In [1]:
import os,sys
import numpy as np
from collections import namedtuple
import tqdm
import glob
import math
import random
import inspect
import os.path as osp
from pathlib import Path
import itertools
from itertools import chain
import numpy as np
import pandas as pd
import multiprocessing
import h5py
import matplotlib.pyplot as plt
import numpy as np
import sys, os
from importlib import reload

import torch
import torch.nn as nn
from torch.utils.data import random_split
from torch_geometric.data import Data, DataLoader, DataListLoader
from torch_geometric.nn import EdgeConv, global_mean_pool, DataParallel
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.transforms as T
from torch_geometric.data import Data,Dataset
from torch_scatter import scatter_mean, scatter
from torch.nn import Sequential as Seq, Linear as Lin, ReLU
from torch_geometric.nn import MetaLayer, EdgeConv, global_mean_pool, DynamicEdgeConv
import h5py

In [24]:
import sys, os
sys.path.append(os.path.abspath(os.path.join('../')))

import models_torch.models as models
reload(models)
import models_torch.losses as losses
reload(losses)
import utils_torch.scaler
reload(utils_torch.scaler)
import utils_torch.preprocessing as prepr
import utils_torch.plot_util as plot
reload(plot)
import utils_torch.train_util as train
reload(train)
import graph_data.graph_data as graph_data
reload(graph_data)
import utils_torch

sys.path.append(os.path.abspath(os.path.join('../../')))
import vande.util.util_plotting as vande_plot
reload(vande_plot)

<module 'vande.util.util_plotting' from '/eos/home-n/nchernya/MLHEP/AnomalyDetection/vande/util/util_plotting.py'>

In [3]:
data_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/'
dataset = graph_data.GraphDataset(root=data_dir,n_jets=1e3)
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
   
use_generator = False
if use_generator:
    validation_split = 0.2
    dataset_size = len(dataset)
    indices = list(range(dataset_size))
    if dataset_size > 2:
        split = int(np.floor(validation_split * dataset_size))
    else: 
        split = 1
    print(dataset_size,split)
    random_seed= 1001

    train_subset, val_subset = torch.utils.data.random_split(dataset, [dataset_size - split, split],
                                                             generator=torch.Generator().manual_seed(random_seed))
    print("train subset dim:", len(train_subset))
    print("validation subset dim", len(val_subset))
    dataloaders = {
        'train':  DataLoader(train_subset, batch_size=128, shuffle=True),
        'val':   DataLoader(val_subset, batch_size=128, shuffle=True)
    }
    print("train_dataloader dim:", len(dataloaders['train']))
    print("val dataloader dim:", len(dataloaders['val']))
else : 
    in_memory_datas = dataset.return_inmemory_data() 

In [ ]:
pf_cands,jet_prop = dataset.pf_cands,dataset.jet_prop
pf_cands_t = torch.cat([torch.tensor(pf_cands[i], dtype=torch.float) for i in range(len(pf_cands))])
#Plot consistuents and jet features prepared for the graph! (but before any normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
jet_feats = 'N_constituents,M,Pt,Eta,Phi'.split(',')


save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
vande_plot.plot_features(pf_cands_t.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)
vande_plot.plot_features(jet_prop[:,0:-1], jet_feats ,'Normalized' , 'QCD', plotname='{}plot_jet_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [4]:
#scaler = standardize(train_subset) # I dont think that this works for the dataset implementation as it is done now, implement scaler as part of dataset ? 
scaler = prepr.standardize(in_memory_datas,minmax_idx=[3,4],log_idx=[3,4]) 

dataloaders = {
    'train':  DataLoader(in_memory_datas, batch_size=128, shuffle=True)
    }
print("train_dataloader dim:", len(dataloaders['train']))

#dataset.get(0).u

train_dataloader dim: 8


In [ ]:
pf_cands_norm = torch.cat([torch.tensor(in_memory_datas[i].x, dtype=torch.float) for i in range(len(in_memory_datas))])
#Plot consistuents and jet features prepared for the graph! (after normalization)
pf_feats = 'px,py,pz,E,pt,eta,phi'.split(',')
save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats_normalized/'
vande_plot.plot_features(pf_cands_norm.numpy(), pf_feats ,'Normalized' , 'QCD', plotname='{}plot_pf_feats_{}'.format(save_dir,'QCD_side'), legend=['QCD'], ylogscale=True)


In [5]:
torch.manual_seed(0)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
multi_gpu = False #torch.cuda.device_count()>1

In [50]:
# model
input_dim = 7
output_dim = 7#4#4,7
big_dim = 32
hidden_dim = 2
#model = models.EdgeNet(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)
model = models.EdgeNetVAE(input_dim=input_dim,output_dim=output_dim, big_dim=big_dim, hidden_dim=hidden_dim)

optimizer = torch.optim.Adam(model.parameters(), lr = 10e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=4, threshold=1e-6)

model.to(device)
summary.gnn_model_summary(model)

EdgeNetVAE(
  (mu_layer): Linear(in_features=32, out_features=2, bias=True)
  (var_layer): Linear(in_features=32, out_features=2, bias=True)
  (batchnorm): BatchNorm1d(7, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (encoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=14, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
  ))
  (decoder): EdgeConv(nn=Sequential(
    (0): Linear(in_features=4, out_features=32, bias=True)
    (1): ReLU()
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=7, bias=True)
  ))
)

In [53]:
#loss
#loss_ftn_obj = losses.LossFunction('mse_coordinates', device=device)
#loss_ftn_obj = losses.LossFunction('mse', device=device)
loss_ftn_obj = losses.LossFunction('vae_loss_mse',beta=0.00001,device=device)

# Training loop
n_epochs = 80
stale_epochs = 0
loss = 999999
#train_losses = []
for epoch in range(0, n_epochs):
    #loss = train(model, optimizer, loader, len(datas), 128, loss_ftn_obj)
    loss_tot,loss_reco,loss_kl = train.train(model, optimizer, dataloaders['train'], len(dataloaders['train'].dataset), dataloaders['train'].batch_size, loss_ftn_obj)
    #train_losses.append(loss)
    print('Epoch: {:02d}, Training Loss Tot, Reco, KL :  {:.4f},{:.4f}, {:.4f}'.format(epoch, loss_tot,loss_reco,loss_kl))









  0%|          | 0/7.8125 [00:00<?, ?it/s]







train loss tot = 0.1547578,train loss reco = 0.0820505,train loss kl = 7270.8125000:   0%|          | 0/7.8125 [00:00<?, ?it/s]







train loss tot = 0.1547578,train loss reco = 0.0820505,train loss kl = 7270.8125000:   0%|          | 0/7.8125 [00:00<?, ?it/s]







train loss tot = 0.1452211,train loss reco = 0.0677557,train loss kl = 7746.6074219:   0%|          | 0/7.8125 [00:00<?, ?it/s]







train loss tot = 0.1452211,train loss reco = 0.0677557,train loss kl = 7746.6074219:   0%|          | 0/7.8125 [00:00<?, ?it/s]







train loss tot = 0.1452211,train loss reco = 0.0677557,train loss kl = 7746.6074219:  26%|██▌       | 2/7.8125 [00:00<00:00, 16.44it/s]







train loss tot = 0.1545799,train loss reco = 0.0825530,train loss kl = 7202.7729492:  26%|██▌       | 2/7.8125 [00:00<00:00, 16.44it/s]







train loss tot = 0.1545799,train loss reco = 0.0825530,train loss kl = 7202.7729492:  26%|██▌       | 2/7.8125 [00:0

Epoch: 00, Training Loss Tot, Reco, KL :  0.1485,0.0738, 7468.1301










train loss tot = 0.1693731,train loss reco = 0.0903984,train loss kl = 7897.5664062:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.01it/s]







train loss tot = 0.1693731,train loss reco = 0.0903984,train loss kl = 7897.5664062:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.01it/s]







train loss tot = 0.1484436,train loss reco = 0.0764736,train loss kl = 7197.0742188:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.01it/s]







train loss tot = 0.1484436,train loss reco = 0.0764736,train loss kl = 7197.0742188:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.01it/s]







train loss tot = 0.1484436,train loss reco = 0.0764736,train loss kl = 7197.0742188:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.18it/s]







train loss tot = 0.1683607,train loss reco = 0.0920773,train loss kl = 7628.4370117:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.18it/s]







train loss tot = 0.1683607,train loss reco = 0.0920773,train loss kl = 7628.4370117:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.18it/s

Epoch: 01, Training Loss Tot, Reco, KL :  0.1550,0.0795, 7553.9914










train loss tot = 0.1565906,train loss reco = 0.0769521,train loss kl = 7963.9287109:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.70it/s]







train loss tot = 0.1565906,train loss reco = 0.0769521,train loss kl = 7963.9287109:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.70it/s]







train loss tot = 0.1494419,train loss reco = 0.0673677,train loss kl = 8207.4892578:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.70it/s]







train loss tot = 0.1494419,train loss reco = 0.0673677,train loss kl = 8207.4892578:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.70it/s]







train loss tot = 0.1494419,train loss reco = 0.0673677,train loss kl = 8207.4892578:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.04it/s]







train loss tot = 0.1661202,train loss reco = 0.0876312,train loss kl = 7848.9902344:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.04it/s]







train loss tot = 0.1661202,train loss reco = 0.0876312,train loss kl = 7848.9902344:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.04it/s

...

Epoch: 77, Training Loss Tot, Reco, KL :  0.1492,0.0740, 7522.9984










train loss tot = 0.1544638,train loss reco = 0.0787601,train loss kl = 7570.4482422:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.65it/s]







train loss tot = 0.1544638,train loss reco = 0.0787601,train loss kl = 7570.4482422:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.65it/s]







train loss tot = 0.1436546,train loss reco = 0.0667948,train loss kl = 7686.0498047:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.65it/s]







train loss tot = 0.1436546,train loss reco = 0.0667948,train loss kl = 7686.0498047:  38%|███▊      | 3/7.8125 [00:00<00:00, 23.65it/s]







train loss tot = 0.1436546,train loss reco = 0.0667948,train loss kl = 7686.0498047:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.03it/s]







train loss tot = 0.1572768,train loss reco = 0.0722046,train loss kl = 8507.2900391:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.03it/s]







train loss tot = 0.1572768,train loss reco = 0.0722046,train loss kl = 8507.2900391:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.03it/s

Epoch: 78, Training Loss Tot, Reco, KL :  0.1487,0.0749, 7376.7158










train loss tot = 0.1477637,train loss reco = 0.0749127,train loss kl = 7285.1669922:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.02it/s]







train loss tot = 0.1477637,train loss reco = 0.0749127,train loss kl = 7285.1669922:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.02it/s]







train loss tot = 0.1472266,train loss reco = 0.0663642,train loss kl = 8086.3076172:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.02it/s]







train loss tot = 0.1472266,train loss reco = 0.0663642,train loss kl = 8086.3076172:  38%|███▊      | 3/7.8125 [00:00<00:00, 24.02it/s]







train loss tot = 0.1472266,train loss reco = 0.0663642,train loss kl = 8086.3076172:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.17it/s]







train loss tot = 0.1427173,train loss reco = 0.0673612,train loss kl = 7535.6835938:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.17it/s]







train loss tot = 0.1427173,train loss reco = 0.0673612,train loss kl = 7535.6835938:  77%|███████▋  | 6/7.8125 [00:00<00:00, 24.17it/s

Epoch: 79, Training Loss Tot, Reco, KL :  0.1445,0.0688, 7575.2796


In [54]:
reload(utils_torch.scaler)
reload(prepr)
reload(plot)

save_dir = '/eos/user/n/nchernya/MLHEP/AnomalyDetection/ADgvae/output_models/pytroch/input_feats/'
inverse_standardization = True
plot.plot_reco_for_loader(model, dataloaders['train'], device, scaler, inverse_standardization, 'test_train', osp.join(save_dir, 'reconstruction_post_train', 'train_reco_std_log_minmax_vae'), 'all')


In [34]:
input_fts, reco_fts = plot.gen_in_out(model, dataloaders['train'], device)


In [64]:
def gnn_model_summary(model):
    
    model_params_list = list(model.named_parameters())
    print("----------------------------------------------------------------")
    line_new = "{:>20}  {:>25} {:>15}".format("Layer.Parameter", "Param Tensor Shape", "Param #")
    print(line_new)
    print("----------------------------------------------------------------")
    for elem in model_params_list:
        p_name = elem[0] 
        p_shape = list(elem[1].size())
        p_count = torch.tensor(elem[1].size()).prod().item()
        line_new = "{:>20}  {:>25} {:>15}".format(p_name, str(p_shape), str(p_count))
        print(line_new)
    print("----------------------------------------------------------------")
    total_params = sum([param.nelement() for param in model.parameters()])
    print("Total params:", total_params)
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params:", num_trainable_params)
    print("Non-trainable params:", total_params - num_trainable_params)

In [65]:
gnn_model_summary(model)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
     mu_layer.weight                    [2, 32]              64
       mu_layer.bias                        [2]               2
    var_layer.weight                    [2, 32]              64
      var_layer.bias                        [2]               2
    batchnorm.weight                        [7]               7
      batchnorm.bias                        [7]               7
 encoder.nn.0.weight                   [32, 14]             448
   encoder.nn.0.bias                       [32]              32
 encoder.nn.2.weight                   [32, 32]            1024
   encoder.nn.2.bias                       [32]              32
 decoder.nn.0.weight                    [32, 4]             128
   decoder.nn.0.bias                       [32]              32
 decoder.nn.2.weight                  